In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
train=os.path.join("F:/Dataset/facial/images/train")
train_files=os.listdir(train)
print(train_files)
print("Validation Data")
val=os.path.join("F:/Dataset/facial/images/validation")
val_files=os.listdir(val)
print(val_files)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Validation Data
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [3]:
train_angry=os.path.join("F:/Dataset/facial/images/train/angry")
print("Total Training Files for angry",len(os.listdir(train_angry)))
train_disgust=os.path.join("F:/Dataset/facial/images/train/disgust")
print("Total Training Files for disgust",len(os.listdir(train_disgust)))
train_fear=os.path.join("F:/Dataset/facial/images/train/fear")
print("Total Training Files for fear",len(os.listdir(train_fear)))
train_happy=os.path.join("F:/Dataset/facial/images/train/happy")
print("Total Training Files for happy",len(os.listdir(train_happy)))
train_neutral=os.path.join("F:/Dataset/facial/images/train/neutral")
print("Total Training Files for neutral",len(os.listdir(train_neutral)))
train_sad=os.path.join("F:/Dataset/facial/images/train/sad")
print("Total Training Files for sad",len(os.listdir(train_sad)))
train_surprise=os.path.join("F:/Dataset/facial/images/train/surprise")
print("Total Training Files for surprise",len(os.listdir(train_surprise)))

Total Training Files for angry 3993
Total Training Files for disgust 436
Total Training Files for fear 4103
Total Training Files for happy 7164
Total Training Files for neutral 4982
Total Training Files for sad 4938
Total Training Files for surprise 3205


In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = 'F:/Dataset/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (75, 75,3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
  
pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 37, 37, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 37, 37, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 37, 37, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [7]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.4)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (7, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 37, 37, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 37, 37, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 37, 37, 32)   0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [9]:
train_dir='F:/Dataset/facial/images/train'
validation_dir='F:/Dataset/facial/images/validation'

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator( rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'categorical', 
                                                    target_size = (75, 75))   

validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'categorical', 
                                                          target_size = (75, 75))

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [12]:
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 30,
            validation_steps = 50,
            verbose = 2)

model.save("face.h5")

Epoch 1/30
100/100 - 168s - loss: 1.4491 - acc: 0.4625 - val_loss: 3.9219 - val_acc: 0.3460
Epoch 2/30
100/100 - 165s - loss: 1.4113 - acc: 0.4780 - val_loss: 4.0455 - val_acc: 0.3490
Epoch 3/30
100/100 - 165s - loss: 1.4238 - acc: 0.4735 - val_loss: 4.3047 - val_acc: 0.3480
Epoch 4/30
100/100 - 166s - loss: 1.4157 - acc: 0.4660 - val_loss: 3.9986 - val_acc: 0.3620
Epoch 5/30
100/100 - 165s - loss: 1.4122 - acc: 0.4590 - val_loss: 4.1856 - val_acc: 0.3530
Epoch 6/30
100/100 - 165s - loss: 1.4283 - acc: 0.4679 - val_loss: 4.3226 - val_acc: 0.3470
Epoch 7/30
100/100 - 166s - loss: 1.4695 - acc: 0.4630 - val_loss: 4.7200 - val_acc: 0.3600
Epoch 8/30
100/100 - 171s - loss: 1.4059 - acc: 0.4925 - val_loss: 5.0427 - val_acc: 0.3490
Epoch 9/30
100/100 - 165s - loss: 1.3800 - acc: 0.4880 - val_loss: 4.7924 - val_acc: 0.3490
Epoch 10/30
100/100 - 165s - loss: 1.3886 - acc: 0.4700 - val_loss: 5.5497 - val_acc: 0.3280
Epoch 11/30
100/100 - 165s - loss: 1.3712 - acc: 0.4950 - val_loss: 5.0344 - va

In [1]:
from tensorflow.keras.models import load_model

classifier = load_model('face.h5')

In [10]:
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        #color_mode = 'grayscale',
        target_size=(75, 75),
        batch_size=20,
        class_mode='categorical',)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
print(class_labels)

Found 7066 images belonging to 7 classes.
{0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}


In [11]:
from tensorflow.keras.models import load_model
classifier = load_model('final_xception.h5')
import cv2
import numpy as np
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array

face_classifier = cv2.CascadeClassifier('C:/Users/KHAN/Anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')

def face_detector(img):
    #Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces = face_classifier.detectMultiScale(gray)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]

    try:
        roi_gray = cv2.resize(roi_gray, (48, 48))
    except:
        return (x,w,y,h), np.zeros((48,48), np.uint8), img
    return (x,w,y,h), roi_gray, img

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    rect, face, image = face_detector(frame)
    if np.sum([face]) != 0.0:
        roi = face.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        # make a prediction on the ROI, then lookup the class
        preds = classifier.predict(roi)[0]
        label = class_labels[preds.argmax()]  
        label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
        cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
    else:
        cv2.putText(image, "No Face Found", (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
        
    cv2.imshow('All', image)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()